# Prediction Insights

## Overview

Howso Engine enables powerful predictions with complete attribution and detailed explanations to make learning from
and debugging your data and predictions as easy as possible. For more information on predictions with Howso Engine,
check out the [predictions user guide](https://docs.howso.com/user_guide/basic_capabilities/predictions.html).

In [1]:
from pprint import pprint

import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from howso.engine import Trainee
from howso.utilities import infer_feature_attributes

## Setup

This recipe will focus on the insights gained from predictions. Specifically, insights from:

- Influential Cases,
- Feature Contributions,
- Residual and Similarity Conviction,
- and Categorical Action Probabilities.

The [basic workflow guide](https://docs.howso.com/user_guide/basic_capabilities/basic_workflow.html) covers creating a Trainee and performing a basic react in detail.

### Load Data and Create Trainee

In [2]:
df = pd.read_csv("../../data/iris/iris.tsv.gz", sep="\t", compression="gzip")
train_data = df.iloc[:-30]
new_data = df[~df.index.isin(train_data.index)]
features = infer_feature_attributes(train_data)

t = Trainee(features=features)

df

Version 29.0.0 of Howso Engine™ is available. You are using version 28.0.4.dev3+g0840d31.
The following parameters from your configuration will override the default Amalgam parameters: {'library_path'}


,Unnamed: 0,sepal length,sepal width,petal length,petal width,class
0,27,5.2,3.5,1.5,0.2,Iris-setosa
1,14,5.8,4.0,1.2,0.2,Iris-setosa
2,25,5.0,3.0,1.6,0.2,Iris-setosa
3,82,5.8,2.7,3.9,1.2,Iris-versicolor
4,97,6.2,2.9,4.3,1.3,Iris-versicolor
...,...,...,...,...,...,...
145,126,6.2,2.8,4.8,1.8,Iris-virginica
146,124,6.7,3.3,5.7,2.1,Iris-virginica
147,78,6.0,2.9,4.5,1.5,Iris-versicolor
148,125,7.2,3.2,6.0,1.8,Iris-virginica


### Train, Analyze, and React

In [3]:
t.train(train_data)
t.analyze()
t.react_into_features(similarity_conviction=True)

reaction = t.react(
    contexts=new_data,
    context_features=["sepal length", "sepal width", "petal length", "petal width"],
    action_features=["class"],
    details={
        "influential_cases": True,
        "similarity_conviction": True,
        "feature_contributions_robust": True,
        "feature_residuals_robust": True,
        "case_feature_residual_convictions_robust": True,
        "categorical_action_probabilities": True,
    }
)

Note that, unlike in the basic workflow guide, we include several `details` in the react call and predict multiple cases as opposed to one. These are what will enable the
insights that we're going to get after the predictions are made.

For more information, see [the API documentation for `Trainee.react()`](https://docs.howso.com/api_reference/_autosummary/howso.engine.html#howso.engine.Trainee.react).

### Inspect the Predictions

Howso Engine has high accuracy even on small datasets.

Here we see a comparison of the trained data (dots) and the predicted data (stars). This plot shows that the predictions align well with the trained data.

In [4]:
train_data = train_data.astype({"class": str})
predicted_data = pd.concat([new_data.reset_index(drop=True).drop(columns="class"), reaction["action"]], axis=1)
predicted_data = predicted_data.astype({"class": str})

cmap = px.colors.qualitative.D3
fig = go.Figure()
for i, (label, group) in enumerate(train_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        name=label,
        marker=dict(color=cmap[i], opacity=0.75),
        legendgroup="trained",
        legendgrouptitle_text="Trained class",
    ))

for i, (label, group) in enumerate(predicted_data.groupby("class")):
    fig.add_trace(go.Scatter(
        x=group["petal length"],
        y=group["petal width"],
        mode="markers",
        marker=dict(size=12, symbol="star", color=cmap[i], opacity=0.75),
        name=label,
        legendgroup="predicted",
        legendgrouptitle_text="Predicted class",
        hovertext=group.index,
    ))

fig.update_layout(
    xaxis_title="Petal Length",
    yaxis_title="Petal Width",
    width=1250,
    title="Trained and Predicted Values"
)
fig.show()

For categorical action features, the prediction can be further understood with the `categorical_action_probabilities` detail.  
This shows the probability of each possible value of the target feature. This information can highlight cases that are on the
border of two classes, like some of the above points are.  The closer a case gets to a class border, the more mixed the
categorical action probabilities may get.

In [5]:
pprint(reaction["details"]["categorical_action_probabilities"], compact=True)

[{'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-versicolor': 0.25952490046481064,
            'Iris-virginica': 0.7404750995351893}},
 {'class': {'Iris-versicolor': 0.10794027518981379,
            'Iris-virginica': 0.8920597248101862}},
 {'class': {'Iris-versicolor': 0.2256381480374078,
            'Iris-virginica': 0.7743618519625921}},
 {'class': {'Iris-versicolor': 0.5287223876453855,
            'Iris-virginica': 0.4712776123546145}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.15892608747806253,
            'Iris-virginica': 0.8410739125219374}},
 {'class': {'Iris-versicolor': 1}}, {'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-virginica': 1}}, {'class': {'Iris-versicolor': 1}},
 {'class': {'Iris-setosa': 1}},
 {'class': {'Iris-versicolor': 0.2803981410398354,
            'Iris-virginica': 0.7196018589601646}},
 {'class': {'Iris-versicolor': 0.9324891884259178,
            'Iris-virginica': 0.06751081157408216}},

For example, if we look at case `4`, we see a mix of probabilities because this case is on the border of two classes.

In [6]:
reaction["details"]["categorical_action_probabilities"][4]

{'class': {'Iris-versicolor': 0.5287223876453855,
  'Iris-virginica': 0.4712776123546145}}

### Insight 1: Which Cases Contributed?

Howso provides complete attribution for any and all predictions, showing exactly which cases influenced each prediction.  This can be used to understand or debug predictions.  For instance,
we can inspect the influential cases for one of the cases (case `4`) that was on the decision boundary in the plot above.  We can see that there is a mix of class values and no cases stand
out in terms of influence weight:

In [7]:
inf_cases_4 = pd.DataFrame(reaction["details"]["influential_cases"][4])
inf_cases_4

,class,sepal length,petal length,.session,petal width,sepal width,.influence_weight,.session_training_index
0,Iris-versicolor,6.1,4.7,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.4,2.9,0.132004,89
1,Iris-versicolor,6.1,4.6,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.4,3.0,0.110184,26
2,Iris-virginica,6.3,5.6,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.8,2.9,0.087547,61
3,Iris-versicolor,6.1,4.7,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.2,2.8,0.076775,100
4,Iris-versicolor,6.3,4.9,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.5,2.5,0.076570,107
5,Iris-virginica,6.3,5.1,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.5,2.8,0.074904,99
6,Iris-virginica,6.4,5.6,2ec65e6b-538d-4706-a7f1-c30bfd950b39,2.1,2.8,0.068011,36
7,Iris-versicolor,6.1,4.0,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.3,2.8,0.067565,30
8,Iris-versicolor,6.8,4.8,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.4,2.8,0.065625,57
9,Iris-virginica,6.0,5.0,2ec65e6b-538d-4706-a7f1-c30bfd950b39,1.5,2.2,0.065129,34


Compare that to the influential cases for case `20`, which is firmly in the center of the solo cluster.  Here we see a single class value.

In [8]:
inf_cases_20 = pd.DataFrame(reaction["details"]["influential_cases"][20])
inf_cases_20

,class,sepal length,petal length,.session,petal width,sepal width,.influence_weight,.session_training_index
0,Iris-setosa,5.0,1.6,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.4,3.4,0.136577,63
1,Iris-setosa,5.4,1.7,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.2,3.4,0.109494,9
2,Iris-setosa,5.1,1.5,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.2,3.4,0.091490,85
3,Iris-setosa,5.1,1.5,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.4,3.7,0.087352,117
4,Iris-setosa,5.0,1.5,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.2,3.4,0.079457,69
5,Iris-setosa,5.4,1.5,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.2,3.7,0.078591,104
6,Iris-setosa,5.4,1.3,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.4,3.9,0.075901,21
7,Iris-setosa,5.4,1.7,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.4,3.9,0.075901,47
8,Iris-setosa,5.2,1.4,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.2,3.4,0.064987,43
9,Iris-setosa,5.2,1.5,2ec65e6b-538d-4706-a7f1-c30bfd950b39,0.2,3.5,0.060319,0


With the influential cases we can derive additional insights, such as identifying anomalous cases from within the influential cases using `similarity_conviction`.  
This can help to identify data that are making predictions more noisy or identify what type of data should be collected to improve predictive power in the future.

In [9]:
inf_case_indices = inf_cases_4[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"]
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
9,6.0,2.2,5.0,1.5,Iris-virginica,0.539284
4,6.3,2.5,4.9,1.5,Iris-versicolor,0.792292
5,6.3,2.8,5.1,1.5,Iris-virginica,0.898854
8,6.8,2.8,4.8,1.4,Iris-versicolor,0.979959
11,6.4,2.7,5.3,1.9,Iris-virginica,1.053584
1,6.1,3.0,4.6,1.4,Iris-versicolor,1.061866
3,6.1,2.8,4.7,1.2,Iris-versicolor,1.098108
2,6.3,2.9,5.6,1.8,Iris-virginica,1.116672
12,6.1,3.0,4.9,1.8,Iris-virginica,1.157878
7,6.1,2.8,4.0,1.3,Iris-versicolor,1.168614


In [10]:
inf_case_indices = inf_cases_20[[".session", ".session_training_index"]].values.tolist()

anom_df = t.get_cases(
    case_indices=inf_case_indices,
    features=["sepal length", "sepal width", "petal length", "petal width", "class", "similarity_conviction"],
)

anom_df.sort_values(by="similarity_conviction")

,sepal length,sepal width,petal length,petal width,class,similarity_conviction
7,5.4,3.9,1.7,0.4,Iris-setosa,0.788960
6,5.4,3.9,1.3,0.4,Iris-setosa,0.862912
11,4.6,3.4,1.4,0.3,Iris-setosa,0.909101
10,5.3,3.7,1.5,0.2,Iris-setosa,0.924549
0,5.0,3.4,1.6,0.4,Iris-setosa,0.966215
1,5.4,3.4,1.7,0.2,Iris-setosa,1.058567
3,5.1,3.7,1.5,0.4,Iris-setosa,1.096774
5,5.4,3.7,1.5,0.2,Iris-setosa,1.127369
2,5.1,3.4,1.5,0.2,Iris-setosa,1.259098
12,5.1,3.5,1.4,0.3,Iris-setosa,1.334568


### Insight 2: Which Features Contributed?

In addition to providing attribution to cases, Howso also provides robust feature contributions to explain which features contributed to each prediction.
When inspecting the feature importances for case `13`,  we see that `petal width` and `petal length` provided the majority of the contribution to the 
prediction, whereas for case `20` `petal width` and `petal length` only have slightly higher contributions than the other features.  This could indicate 
that different features are more important for predicting different classes within this dataset and that focusing on those features would be prudent.

Identifying that certain cases that influenced a prediction are anomalous can help to explain low predictive power in certain parts of the data or other
issues that may appear during deployment.

In [11]:
fcs_13 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][13:14], index=[13])
fcs_20 = pd.DataFrame(reaction["details"]["feature_contributions_robust"][20:21], index=[20])

display(fcs_13)
display(fcs_20)

,sepal length,petal length,sepal width,petal width
13,0.138392,0.298806,0.069347,0.317151


,sepal length,petal length,sepal width,petal width
20,0.117864,0.157941,0.120061,0.158137


### Insight 3: How Certain is the Prediction?

[Residuals](https://docs.howso.com/user_guide/basic_capabilities/residuals.html) can characterize the uncertainty of the data around the prediction.  This will tell us which features are hard to predict in the region of the data around each case we're predicting.
If a prediction is less accurate than expected, this can explain which features were noisy and may have contributed to the problem.

In [12]:
pd.DataFrame(reaction["details"]["feature_residuals_robust"])

,class,sepal length,petal length,petal width,sepal width
0,0.263076,0.425850,0.336420,0.170437,0.193867
1,0.371544,0.366796,0.444008,0.267980,0.241944
2,0.234404,0.417798,0.468102,0.311137,0.148471
3,0.345302,0.401813,0.439053,0.273151,0.232210
4,0.269957,0.334267,0.398954,0.201734,0.273033
5,0.132105,0.349613,0.327858,0.128720,0.148315
6,0.087075,0.312448,0.409849,0.198073,0.230761
7,0.293749,0.406827,0.481040,0.308434,0.148749
8,0.222520,0.339377,0.338339,0.153270,0.229267
9,0.124605,0.313215,0.353367,0.144614,0.171185


We can also use [`residual conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#prediction-residual-conviction) to determine which features
are uncertain in a scale-invariant manner, which can be useful if you wish to compare different features of different scales against each other.  Higher residual conviction
indicates something that is less surprising than expected and lower redsidual conviction indicates the opposite.

In [13]:
pd.DataFrame(reaction["details"]["case_feature_residual_convictions_robust"])

,class,sepal length,petal length,sepal width,petal width
0,0.935731,1.040432,0.929764,1.954838,1.200450
1,0.919504,1.287464,1.294020,0.599767,0.995986
2,1.053944,0.771654,1.005298,1.943249,0.913768
3,0.946434,0.635264,0.732935,0.556090,0.786650
4,0.539940,1.693306,0.351218,0.728780,0.592054
5,1.375291,1.196381,1.557930,0.873594,2.079028
6,0.587570,1.134108,0.817930,0.733033,1.487699
7,1.412452,1.168242,1.144483,0.675697,2.550670
8,1.864683,0.625757,0.682230,0.362789,0.772524
9,1.440698,0.997258,1.546590,0.427193,0.887897


### Insight 4: How Anomalous are the Predicted Cases?

By getting the [`similarity conviction`](https://docs.howso.com/user_guide/basic_capabilities/conviction.html#similarity-conviction) of the cases that we predict, 
we can determine which of them are anomalous relative to the trained data. This could help to highlight cases that are unusually difficult or easy to predict, such as can be the case
when model drift occurs as a form of model monitoring.

In [14]:
pprint(reaction["details"]["similarity_conviction"], compact=True)

[2.4585196676262817, 1.5357799645929426, 1.970666948810099, 1.1933944706668032,
 0.9051402030417045, 2.6457988977635636, 2.319244637887139, 1.7635392865477066,
 1.1951337317665627, 2.473030947584981, 3.362152446963219, 2.922814248244336,
 4.4404425905247935, 1.6875375040323766, 1.6871073004227093, 1.9153302133705288,
 1.8497021169148855, 2.9722704787943224, 1.0425530845279973, 0.8458198962301933,
 1.8790664748703299, 2.2923283080911636, 1.0554025969482528, 2.903016590748419,
 3.17077924543639, 2.0846188699332453, 2.0056603024939266, 2.3093496485719256,
 1.3953345053263322, 1.2405275771206232]


In this case, there are no cases with a low similarity_conviction which indicates that there are no cases that we reacted to which are particularly
anomalous.